In [3]:
import tensorflow as tf
import os

os.chdir("C:/nt/AIOps/RRU_CLASSIFICATION/research")
os.chdir("../")
print(os.getcwd())

C:\nt\AIOps\RRU_CLASSIFICATION


In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/c17hawke/FSDS_NOV_deepCNNClassifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="c17hawke"
os.environ["MLFLOW_TRACKING_PASSWORD"]="84215e85b5e87347572d9d272c798b2b1ff2a546"

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    best_model_path:Path

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.training.trained_model_path,
            training_data = os.path.join(self.config.data_ingestion.unzip_dir, self.config.evaluation.artifact_dir),
            best_model_path= self.config.evaluation.best_model_path,
            # training_data=self.config.data_ingestion.unzip_dir,
            mlflow_uri="https://dagshub.com/c17hawke/FSDS_NOV_deepCNNClassifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from deepClassifier.entity import EvaluationConfig
from deepClassifier.utils import save_json,load_json
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix,classification_report

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            # validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            # subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.pred = self.model.predict(self.valid_generator)
        y=np.concatenate([self.valid_generator.next()[1] for i in range(self.valid_generator.__len__())])
        predicted = np.argmax(self.pred, axis=1)
        
        report = classification_report(np.argmax(y, axis=1), predicted)
        self.precision_score = precision_score(np.argmax(y, axis=1), predicted, average="macro")
        self.recall_score = recall_score(np.argmax(y, axis=1), predicted, average="macro")
        self.f1_score = f1_score (np.argmax(y, axis=1), predicted , average="macro")
        print(report)
        
        if Path("scores.json").exists():
            best_score = self.load_score(path=Path("scores.json"))
            if best_score['f1_score']>self.f1_score:
                self.save_score()
                self.save_model(path=Path(self.config.best_model_path))
        else:
            self.save_score()
            self.save_model(path=Path(self.config.best_model_path))
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def load_score(self, path: Path):
        return load_json(path)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1],
                 "precision_score":self.precision_score,"recall_score":self.recall_score,"f1_score":self.f1_score}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    # evaluation.save_score()
    # evaluation.log_into_mlflow()
    
except Exception as e:
   raise e

Found 146 images belonging to 2 classes.
5/5 [==============================] - 23s 4s/step
              precision    recall  f1-score   support

           0       0.76      0.87      0.81        75
           1       0.84      0.72      0.77        71

    accuracy                           0.79       146
   macro avg       0.80      0.79      0.79       146
weighted avg       0.80      0.79      0.79       146



TypeError: save_model() missing 1 required positional argument: 'model'